# Phase 3 Weighted Bagging

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import os
import re
import csv
import codecs
import gensim
import itertools
import numpy as np
import pandas as pd
import operator
import sys

from nltk import ngrams
from collections import Counter
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from iwillwin.data_utils.feature_engineering import FeatureCreator

from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize

import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
from os import listdir
from os.path import isfile, join

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from keras.utils import to_categorical

C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS, MAX_SEQUENCE_LENGTH = 50000, 50
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains_nns, tests_nns, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zake7\AppData\Local\Temp\jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built succesfully.


[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [3]:
trains_meta = trains_nns[2]
tests_meta = tests_nns[2]

In [4]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [5]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [6]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains_nns[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests_nns[2], test_has_rumor.reshape((-1, 1))), axis=1)

In [7]:
oof_file_names = sorted([f for f in listdir('../data/oofs/') if isfile(join('../data/oofs/', f)) and f != '.gitkeep'])
preds_file_names = [name.replace('-Train', '') for name in oof_file_names]

oofs = []
preds = []
for name in oof_file_names:
    oofs.append(pd.read_csv('../data/oofs/' + name))
for name in preds_file_names:
    preds.append(pd.read_csv('../data/output/' + name))    

In [8]:
for i, name in enumerate(oof_file_names):
    print(i, name)
    
trains = pd.DataFrame()
tests = pd.DataFrame()

for i in range(len(oof_file_names)):
    for label_type in ['agreed', 'disagreed', 'unrelated']:
        trains['oofs_{}_{}'.format(i, label_type)] = oofs[i][label_type].values
        tests['oofs_pred{}_{}'.format(i, label_type)] = preds[i][label_type].values

0 3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-NoEM-Train-L0.809633-NB5000.csv
1 3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.816583-NB5000.csv
2 3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.833565-NB5000.csv
3 WordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.838202-NB100000.csv
4 WordSGNS-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L1.104962-NB100000.csv
5 WordTC-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.8440-NB100000.csv
6 WordTC-DenseRNN-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.854586-NB100000.csv
7 WordTC-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.374334-NB100000.csv
8 WordTC-Gated4GWindows-NoMeta-3P-NoEM-NoClassWeighted-3Layers-withEM-Train-L0.836860-NB100000.csv


In [9]:
unrelated = pd.DataFrame()
agreeds = pd.DataFrame()
disagreeds = pd.DataFrame()

#check_oofs = True
check_oofs = False


if check_oofs:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = oofs[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = oofs[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = oofs[i]['disagreed'].values
else:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = preds[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = preds[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = preds[i]['disagreed'].values  

In [10]:
agreeds.corr()

,oofs_agreed_0,oofs_agreed_1,oofs_agreed_2,oofs_agreed_3,oofs_agreed_4,oofs_agreed_5,oofs_agreed_6,oofs_agreed_7,oofs_agreed_8
oofs_agreed_0,1.000000,0.982310,0.976632,0.959797,0.956807,0.961449,0.961597,0.959404,0.962629
oofs_agreed_1,0.982310,1.000000,0.982479,0.958966,0.957473,0.961494,0.961780,0.961639,0.961288
oofs_agreed_2,0.976632,0.982479,1.000000,0.955476,0.959167,0.957512,0.961050,0.963775,0.956841
oofs_agreed_3,0.959797,0.958966,0.955476,1.000000,0.982100,0.977313,0.970090,0.972992,0.973543
oofs_agreed_4,0.956807,0.957473,0.959167,0.982100,1.000000,0.971693,0.972289,0.979278,0.968705
oofs_agreed_5,0.961449,0.961494,0.957512,0.977313,0.971693,1.000000,0.981701,0.981716,0.982047
oofs_agreed_6,0.961597,0.961780,0.961050,0.970090,0.972289,0.981701,1.000000,0.983917,0.982819
oofs_agreed_7,0.959404,0.961639,0.963775,0.972992,0.979278,0.981716,0.983917,1.000000,0.978351
oofs_agreed_8,0.962629,0.961288,0.956841,0.973543,0.968705,0.982047,0.982819,0.978351,1.000000


In [11]:
disagreeds.corr()

,oofs_disagreeds_0,oofs_disagreeds_1,oofs_disagreeds_2,oofs_disagreeds_3,oofs_disagreeds_4,oofs_disagreeds_5,oofs_disagreeds_6,oofs_disagreeds_7,oofs_disagreeds_8
oofs_disagreeds_0,1.000000,0.954897,0.951416,0.912412,0.913741,0.912573,0.916230,0.912694,0.908535
oofs_disagreeds_1,0.954897,1.000000,0.964977,0.920113,0.918646,0.920680,0.921862,0.920311,0.911443
oofs_disagreeds_2,0.951416,0.964977,1.000000,0.919030,0.925045,0.915579,0.923305,0.926555,0.913171
oofs_disagreeds_3,0.912412,0.920113,0.919030,1.000000,0.967334,0.956456,0.945988,0.954167,0.943892
oofs_disagreeds_4,0.913741,0.918646,0.925045,0.967334,1.000000,0.946770,0.944932,0.961943,0.939010
oofs_disagreeds_5,0.912573,0.920680,0.915579,0.956456,0.946770,1.000000,0.965450,0.960278,0.955281
oofs_disagreeds_6,0.916230,0.921862,0.923305,0.945988,0.944932,0.965450,1.000000,0.963620,0.958197
oofs_disagreeds_7,0.912694,0.920311,0.926555,0.954167,0.961943,0.960278,0.963620,1.000000,0.954127
oofs_disagreeds_8,0.908535,0.911443,0.913171,0.943892,0.939010,0.955281,0.958197,0.954127,1.000000


In [12]:
unrelated.corr()

,oofs_unrelated_0,oofs_unrelated_1,oofs_unrelated_2,oofs_unrelated_3,oofs_unrelated_4,oofs_unrelated_5,oofs_unrelated_6,oofs_unrelated_7,oofs_unrelated_8
oofs_unrelated_0,1.000000,0.978855,0.972572,0.952930,0.949506,0.954771,0.955016,0.952586,0.955786
oofs_unrelated_1,0.978855,1.000000,0.979417,0.952496,0.950368,0.955304,0.955510,0.955456,0.954800
oofs_unrelated_2,0.972572,0.979417,1.000000,0.948158,0.952290,0.950251,0.954365,0.957542,0.949502
oofs_unrelated_3,0.952930,0.952496,0.948158,1.000000,0.979002,0.973876,0.965568,0.969264,0.969451
oofs_unrelated_4,0.949506,0.950368,0.952290,0.979002,1.000000,0.967039,0.967506,0.975886,0.963249
oofs_unrelated_5,0.954771,0.955304,0.950251,0.973876,0.967039,1.000000,0.978773,0.978637,0.978786
oofs_unrelated_6,0.955016,0.955510,0.954365,0.965568,0.967506,0.978773,1.000000,0.981089,0.979624
oofs_unrelated_7,0.952586,0.955456,0.957542,0.969264,0.975886,0.978637,0.981089,1.000000,0.974969
oofs_unrelated_8,0.955786,0.954800,0.949502,0.969451,0.963249,0.978786,0.979624,0.974969,1.000000


# Prepare Different Inputs

In [13]:
# Only use oofs
ensemble_trains = trains.values
ensemble_tests = tests.values

In [45]:
# use oofs and meta-features
#ensemble_trains = np.concatenate((trains.values, trick_trains_features), axis=1)
#ensemble_tests = np.concatenate((tests.values, trick_tests_features), axis=1)

# Ensemble With NN

In [14]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

In [15]:
from keras import regularizers

def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32') # hard 
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

def get_dense_add_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.1)(new_features)
        features = Concatenate()([features, new_features])

    h = Highway(activation='relu')(features)
    out_ = Dense(3, activation='softmax')(h)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [28]:
def get_logit_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")    
    out_ = Dense(3, activation='softmax')(features_inputs)
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [16]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, features, y, fold_count, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        weight_val=scale_sample_weight
        class_weight=None
        fold_size = len(features) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(features)

            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])
            
            val_features = features[fold_start:fold_end]
            val_y = y[fold_start:fold_end]
            fold_pos = (np.sum(train_y) / len(train_features))

            train_data = {
                "mata-features": train_features,
            }

            val_data = {
                "mata-features": val_features,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         callbacks=[early_stopping, model_checkpoint],)
        bst_val_score = max(hist.history['val_weighted_accuracy'])
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [17]:
def _agent_get_model():
    return get_dense_add_net(ensemble_trains.shape[1])

In [24]:
def np_weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    weight_mask = np.max(weight_mask, axis=-1)
    norms = np.sum(weight_mask)
    
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(y_pred, axis=-1)
    
    res = ((y_true == y_pred) * weight_mask).sum() / norms
    return res

In [ ]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32')
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

In [19]:
trainer = KerasModelTrainer(model_stamp="Ensemble-DenseNet", epoch_num=500)
models, score, folds_preds = trainer.train_folds(features=ensemble_trains, y=to_categorical(labels), augments=None, fold_count=10,
    batch_size=1024, 
    scale_sample_weight=None, class_weight=None,
    get_model_func=_agent_get_model, 
    patience=20)

C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 27)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 24)           672         mata-features[0][0]              
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 24)           0           dense_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 51)           0           mata-features[0][0]              
                                                                 dropout_1[0][0]                  
__________

288497/288497 [==============================] - ETA: 2s - loss: 0.2411 - acc: 0.8906 - weighted_accuracy: 0.870 - ETA: 1s - loss: 0.2657 - acc: 0.8824 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2623 - acc: 0.8843 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2613 - acc: 0.8843 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2606 - acc: 0.8847 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2621 - acc: 0.8843 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2635 - acc: 0.8835 - weighted_accuracy: 0.859 - ETA: 0s - loss: 0.2633 - acc: 0.8835 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2640 - acc: 0.8836 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2637 - acc: 0.8839 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2625 - acc: 0.8847 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2619 - acc: 0.8849 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2622 - acc: 0.8847 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2623 - acc: 0.8846 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2623 - ac

288497/288497 [==============================] - ETA: 2s - loss: 0.2611 - acc: 0.8867 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2603 - acc: 0.8848 - weighted_accuracy: 0.857 - ETA: 1s - loss: 0.2594 - acc: 0.8848 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2606 - acc: 0.8833 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2617 - acc: 0.8834 - weighted_accuracy: 0.859 - ETA: 0s - loss: 0.2622 - acc: 0.8836 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2605 - acc: 0.8848 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2602 - acc: 0.8849 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2604 - acc: 0.8848 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2605 - acc: 0.8849 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2601 - acc: 0.8851 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2605 - acc: 0.8848 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2605 - acc: 0.8847 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2610 - acc: 0.8844 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2603 - ac

288497/288497 [==============================] - ETA: 1:50 - loss: 0.9500 - acc: 0.6924 - weighted_accuracy: 0.001 - ETA: 9s - loss: 0.5377 - acc: 0.8421 - weighted_accuracy: 0.6449  - ETA: 5s - loss: 0.4309 - acc: 0.8580 - weighted_accuracy: 0.736 - ETA: 3s - loss: 0.3872 - acc: 0.8650 - weighted_accuracy: 0.775 - ETA: 2s - loss: 0.3636 - acc: 0.8687 - weighted_accuracy: 0.795 - ETA: 2s - loss: 0.3469 - acc: 0.8710 - weighted_accuracy: 0.807 - ETA: 2s - loss: 0.3360 - acc: 0.8724 - weighted_accuracy: 0.815 - ETA: 1s - loss: 0.3264 - acc: 0.8739 - weighted_accuracy: 0.822 - ETA: 1s - loss: 0.3204 - acc: 0.8747 - weighted_accuracy: 0.826 - ETA: 1s - loss: 0.3144 - acc: 0.8760 - weighted_accuracy: 0.830 - ETA: 1s - loss: 0.3093 - acc: 0.8769 - weighted_accuracy: 0.833 - ETA: 1s - loss: 0.3060 - acc: 0.8773 - weighted_accuracy: 0.835 - ETA: 0s - loss: 0.3033 - acc: 0.8776 - weighted_accuracy: 0.837 - ETA: 0s - loss: 0.3003 - acc: 0.8783 - weighted_accuracy: 0.839 - ETA: 0s - loss: 0.2982 

...

288495/288495 [==============================] - ETA: 3s - loss: 0.2718 - acc: 0.8896 - weighted_accuracy: 0.857 - ETA: 1s - loss: 0.2716 - acc: 0.8801 - weighted_accuracy: 0.856 - ETA: 1s - loss: 0.2656 - acc: 0.8817 - weighted_accuracy: 0.857 - ETA: 1s - loss: 0.2631 - acc: 0.8828 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2611 - acc: 0.8832 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2632 - acc: 0.8824 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2632 - acc: 0.8832 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2632 - acc: 0.8833 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2632 - acc: 0.8836 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2625 - acc: 0.8841 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2623 - acc: 0.8844 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2621 - acc: 0.8842 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2620 - acc: 0.8842 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2615 - acc: 0.8846 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2622 - ac

288495/288495 [==============================] - ETA: 2s - loss: 0.2751 - acc: 0.8730 - weighted_accuracy: 0.852 - ETA: 1s - loss: 0.2652 - acc: 0.8813 - weighted_accuracy: 0.857 - ETA: 1s - loss: 0.2623 - acc: 0.8819 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2614 - acc: 0.8839 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2615 - acc: 0.8838 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2596 - acc: 0.8851 - weighted_accuracy: 0.862 - ETA: 0s - loss: 0.2606 - acc: 0.8849 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2619 - acc: 0.8843 - weighted_accuracy: 0.861 - ETA: 0s - loss: 0.2623 - acc: 0.8838 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2629 - acc: 0.8836 - weighted_accuracy: 0.859 - ETA: 0s - loss: 0.2623 - acc: 0.8838 - weighted_accuracy: 0.859 - ETA: 0s - loss: 0.2616 - acc: 0.8839 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2618 - acc: 0.8838 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2618 - acc: 0.8837 - weighted_accuracy: 0.860 - ETA: 0s - loss: 0.2621 - ac

288495/288495 [==============================] - ETA: 4s - loss: 0.2274 - acc: 0.9053 - weighted_accuracy: 0.887 - ETA: 2s - loss: 0.2606 - acc: 0.8865 - weighted_accuracy: 0.863 - ETA: 2s - loss: 0.2636 - acc: 0.8853 - weighted_accuracy: 0.861 - ETA: 2s - loss: 0.2629 - acc: 0.8849 - weighted_accuracy: 0.861 - ETA: 2s - loss: 0.2625 - acc: 0.8854 - weighted_accuracy: 0.861 - ETA: 1s - loss: 0.2626 - acc: 0.8845 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2624 - acc: 0.8843 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2618 - acc: 0.8850 - weighted_accuracy: 0.861 - ETA: 1s - loss: 0.2625 - acc: 0.8841 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2619 - acc: 0.8848 - weighted_accuracy: 0.861 - ETA: 1s - loss: 0.2615 - acc: 0.8846 - weighted_accuracy: 0.861 - ETA: 1s - loss: 0.2610 - acc: 0.8848 - weighted_accuracy: 0.861 - ETA: 1s - loss: 0.2604 - acc: 0.8849 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2606 - acc: 0.8845 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2602 - ac

288495/288495 [==============================] - ETA: 5s - loss: 0.2441 - acc: 0.8945 - weighted_accuracy: 0.876 - ETA: 2s - loss: 0.2593 - acc: 0.8853 - weighted_accuracy: 0.863 - ETA: 2s - loss: 0.2601 - acc: 0.8857 - weighted_accuracy: 0.863 - ETA: 2s - loss: 0.2589 - acc: 0.8861 - weighted_accuracy: 0.863 - ETA: 2s - loss: 0.2581 - acc: 0.8867 - weighted_accuracy: 0.863 - ETA: 1s - loss: 0.2581 - acc: 0.8865 - weighted_accuracy: 0.863 - ETA: 1s - loss: 0.2578 - acc: 0.8866 - weighted_accuracy: 0.863 - ETA: 1s - loss: 0.2586 - acc: 0.8867 - weighted_accuracy: 0.863 - ETA: 1s - loss: 0.2597 - acc: 0.8857 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2591 - acc: 0.8859 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2594 - acc: 0.8860 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2596 - acc: 0.8856 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2599 - acc: 0.8853 - weighted_accuracy: 0.862 - ETA: 1s - loss: 0.2600 - acc: 0.8851 - weighted_accuracy: 0.861 - ETA: 1s - loss: 0.2604 - ac

288495/288495 [==============================] - ETA: 5s - loss: 0.2809 - acc: 0.8828 - weighted_accuracy: 0.857 - ETA: 2s - loss: 0.2674 - acc: 0.8797 - weighted_accuracy: 0.857 - ETA: 2s - loss: 0.2605 - acc: 0.8823 - weighted_accuracy: 0.860 - ETA: 2s - loss: 0.2659 - acc: 0.8795 - weighted_accuracy: 0.856 - ETA: 2s - loss: 0.2651 - acc: 0.8812 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2649 - acc: 0.8816 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2641 - acc: 0.8818 - weighted_accuracy: 0.858 - ETA: 1s - loss: 0.2630 - acc: 0.8824 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2632 - acc: 0.8821 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2628 - acc: 0.8826 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2630 - acc: 0.8823 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2633 - acc: 0.8822 - weighted_accuracy: 0.859 - ETA: 1s - loss: 0.2626 - acc: 0.8825 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2620 - acc: 0.8831 - weighted_accuracy: 0.860 - ETA: 1s - loss: 0.2618 - ac

In [27]:
print("score", score)
oofs_dir = "../data/ensemble/oofs/"
output_dir = "../data/ensemble/preds/"
onehot_pred_dir = "../data/ensemble/nn_one_hot/"

model_submit_prefix = "AddNN-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
train_predicts = np.concatenate(folds_preds, axis=0)
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict({"mata-features": ensemble_tests}, batch_size=128, verbose=1)
    test_predicts_list.append(test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8612014500879486
Predicting training results...
Predicting testing results...
80126/80126 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 11us/step
Predicting labeled testing results...


In [25]:
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

In [26]:
score

0.8612014500879486